# Initialization
This project assumes GitHub and Google Cloud Storage info is stored in environmental variables. This project was developed in Google Colab. Select either 'colab' or 'local' for the environment.

See the README.md in the GitHub repository https://github.com/tristan-day-research/NeuroStorm_seizure_detection for more information on this project and how to use it.

In [ ]:
# _____ IMPORTANT First select envorinment _________

ENVIRONMENT = 'colab'   # Choose local or colab
BRANCH_NAME = 'main'

import os, sys

if ENVIRONMENT == 'colab':
    from google.colab import userdata

    # Retrieve GitHub and GCP credentials from Colab user data
    token = userdata.get('GITHUB_PAT')
    github_email = userdata.get('GITHUB_EMAIL')
    github_username = userdata.get('GITHUB_USER_NAME')

    GCP_EEG_PROJECT_ID = userdata.get('GCP_EEG_PROJECT_ID')
    BUCKET_NAME = os.getenv('GCP_EEG_BUCKET_NAME')

    # Set Git configuration using Python variables
    !git config --global user.email "{github_email}"
    !git config --global user.name "{github_username}"

    # Clone the repository using the token
    !git clone -b {BRANCH_NAME} https://{token}@github.com/tristan-day-research/NeuroStorm_seizure_detection.git

    # Change to correct directory
    %cd /content/NeuroStorm_seizure_detection/
    ! ls
    !git status

    from google.colab import auth
    auth.authenticate_user()

    !gcloud auth login
    !gcloud config set project {GCP_EEG_PROJECT_ID}


if ENVIRONMENT == 'local':
   # First clone repository directly via UI

    %pip install python-dotenv
    from dotenv import load_dotenv
    import os

    load_dotenv(override=True)


    token = os.getenv('GITHUB_PAT')
    github_email = os.getenv('GITHUB_EMAIL')
    github_username = os.getenv('GITHUB_USER_NAME')

    # Set Git configuration using Python variables
    !git config --global user.email "{github_email}"
    !git config --global user.name "{github_username}"


    !python3 -m ensurepip --upgrade
    !python3 -m pip install --upgrade pip

    !pip install torch

%pip install -r requirements.txt -q
os.chdir('/content/NeuroStorm_seizure_detection')


# Determine if a GPU is available and set the device accordingly

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
!nvidia-smi  # Display GPU


In [2]:
# Standard Library Imports
import gc
import json
import time
import glob
import math
import random
import threading
import subprocess
import logging
from collections import OrderedDict
from datetime import datetime
from functools import partial
from warnings import warn

# Third-Party Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
plt.rcParams["font.family"] = "serif"

from tqdm import tqdm

# Machine Learning/Deep Learning Imports
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torch.nn.init import trunc_normal_
from torch.nn.utils import clip_grad_norm_, rnn
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast
from torchvision.transforms import Compose


# Preprocessing
Each EEG raw signal is segemnted into patches of a fixed length. Fast Fourier Transform (FFT) spectra are made from these patches which will be used to train the Vector-Quantized Variational Autoencoder (VQ-VAE).

# Tokenizer

In [8]:
os.chdir('/content/NeuroStorm_seizure_detection')